In [1]:
import cv2
import os
import time
import csv
import warnings
import pickle

import numpy as np
import pandas as pd

import mediapipe as mp
from mediapipe.framework.formats import landmark_pb2

mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


2025-08-18 20:24:53.812426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755519893.866941    4367 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755519893.884885    4367 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1755519893.992680    4367 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755519893.992761    4367 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1755519893.992764    4367 computation_placer.cc:177] computation placer alr

In [2]:
desire_fps = 15
num_frames = 15

original_width, original_height = 500, 500
small_width, small_height = 150, 150

In [ ]:
with open('../model/v3_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [1]:
def check_direction(landmark):
    direction = "Right"
    smallest_value = min(landmark, key=landmark.get)
    highest_value = max(landmark, key=landmark.get)
    
    if smallest_value == "nose":
        direction = 0
    elif highest_value == "nose":
        direction = 1
    else:
        direction = 2
    
    return direction


In [ ]:
camera = cv2.VideoCapture("http://192.168.100.197:5000/video")

frames = []
detected = []
face_direction = []

desire_fps = 5
frame_delay = 1.0 / desire_fps

if not camera.isOpened():
    print("Error: Could not open camera.")
else:
    print("Camera is ready. Press 'q' to quit.")

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    start_time = time.time()

    while camera.isOpened():
        delay_frame_time = time.time()
        ret, frame = camera.read()

        if ret:

            frame = cv2.resize(frame, (original_width, original_height))

            if len(frames) < num_frames:
                frames.append(cv2.resize(frame, (small_width, small_height)))

                frame_detected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_detected.flags.writeable = False

                results = holistic.process(frame_detected)

                frame_detected.flags.writeable = True
                frame_detected = cv2.cvtColor(frame_detected, cv2.COLOR_RGB2BGR)

                nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
                ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
                ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
                wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
                wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]

                temporary_list = []
                temporary_list.append(nose)
                temporary_list.append(ear_l)
                temporary_list.append(ear_r)
                temporary_list.append(wrist_r)
                temporary_list.append(wrist_l)
                
                face_direction.append(check_direction({"nose": nose.x, "ear_r": ear_r.x, "ear_l": ear_l.x}))
                detected.append(temporary_list)
            else:
                frames.pop(0)
                face_direction.pop(0)
                detected.pop(0)

                frames.append(cv2.resize(frame, (small_width, small_height)))

                frame_detected = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_detected.flags.writeable = False

                results = holistic.process(frame_detected)
                
                frame_detected.flags.writeable = True
                frame_detected = cv2.cvtColor(frame_detected, cv2.COLOR_RGB2BGR)
                
                nose = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.NOSE]
                ear_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_EAR]
                ear_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.LEFT_EAR]
                wrist_r = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
                wrist_l = results.pose_landmarks.landmark[mp.solutions.holistic.PoseLandmark.RIGHT_WRIST]
                
                temporary_list = []
                temporary_list.append(nose)
                temporary_list.append(ear_l)
                temporary_list.append(ear_r)
                temporary_list.append(wrist_r)
                temporary_list.append(wrist_l)
                
                face_direction.append(check_direction({"nose": nose.x, "ear_r": ear_r.x, "ear_l": ear_l.x}))
                detected.append(temporary_list)
            
            if len(frames) >= num_frames:
                stacked1_frames = np.vstack((frames[0], frames[1], frames[2]))
                stacked2_frames = np.vstack((frames[3], frames[4], frames[5]))
                stacked3_frames = np.vstack((frames[6], frames[7], frames[8]))
                stacked4_frames = np.vstack((frames[9], frames[10], frames[11]))
                stacked5_frames = np.vstack((frames[12], frames[13], frames[14]))

                stacked1_frames = cv2.resize(stacked1_frames, (small_width, original_height))
                stacked2_frames = cv2.resize(stacked2_frames, (small_width, original_height))
                stacked3_frames = cv2.resize(stacked3_frames, (small_width, original_height))
                stacked4_frames = cv2.resize(stacked4_frames, (small_width, original_height))
                stacked5_frames = cv2.resize(stacked5_frames, (small_width, original_height))
                
                final_layout = np.hstack((frame, stacked1_frames, stacked2_frames, stacked3_frames, stacked4_frames, stacked5_frames))
            else:
                final_layout = frame

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            flatten_detected = [landmark for sublist in detected for landmark in sublist]

            landmark_list = landmark_pb2.NormalizedLandmarkList()
            landmark_list.landmark.extend(flatten_detected)


            try:
                lv = landmark_list.landmark
                counter = 0
                featureper_frames = 0
                motion_row = []
                
                for lndmrk in lv:
                    motion_row.append(lndmrk.x)
                    motion_row.append(lndmrk.y)
                    motion_row.append(lndmrk.z)
                    motion_row.append(lndmrk.visibility)
                    counter += 1

                    if counter % 5 == 0:
                        motion_row.append(face_direction[featureper_frames])
                        featureper_frames += 1 

                motion_row = list(np.array(motion_row))
                
                motion_detected = pd.DataFrame([motion_row])
                motion_class = model.predict(motion_detected)[0]

                warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
                
                cv2.putText(final_layout, f'Class: {motion_class}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            except Exception as e:
                pass


            cv2.imshow("Camera Feed", final_layout)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
camera.release()
cv2.destroyAllWindows()



Camera is ready. Press 'q' to quit.


MESA: error: ZINK: failed to choose pdev
libEGL warning: egl: failed to create dri2 screen
I0000 00:00:1755519955.006973    4367 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1755519955.044500   17859 gl_context.cc:369] GL version: 3.1 (OpenGL ES 3.1 Mesa 24.0.9-0ubuntu0.3), renderer: D3D12 (AMD Radeon(TM) Graphics)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1755519955.462796   17853 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755519955.642318   17848 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755519955.663884   17850 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1755519955.664125   17842 infer